<a href="https://colab.research.google.com/github/mnassar/py2vpy3xai/blob/main/py2cpy3XAILIMEScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

@author: mnassar

This is the script used to generate the results for the LIME text explainer

Table 9: Results for LIMETextExplainer with r"\s+" split
expression

In [ ]:
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283835 sha256=ace5569a61da2a965318910bca2b1bbce2c1c497b2c3b083d64a4fdc8116816f
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [ ]:
%%time

# -*- coding: utf-8 -*-
"""py2vpy3XaiLime.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1ZMAQnBWghHG5OK5wVofxT41DSlKY6rMJ

Shap
"""

from keras import models
from keras.utils import pad_sequences

import lime
from lime.lime_text import LimeTextExplainer

import numpy as np
import pandas as pd
import re


def predict (lines, shape = 1):
  samples = [  bytearray(line, encoding="utf-8") for line in lines ]
  samples_padded = pad_sequences(samples, maxlen=MAX_LEN, padding='post', truncating='post')
  preds = m.predict(samples_padded)
  # if pred < 0.5:
  #   label="python2"
  # else:
  #   label="python3"
  if shape == 2:
    return np.concatenate ([1-preds, preds], axis=1)
  else:
    return preds


def explain(line):
  exp = explainer.explain_instance(line, lambda x: predict(x, shape=2), num_features=1,
                                   num_samples=32*10)
  return exp.as_list()[0][0]

#longest common subsequence
def lcs(X, Y):
    # find the length of the strings
    m = len(X)
    n = len(Y)
    # declaring the array for storing the dp values
    L = [[None]*(n + 1) for i in range(m + 1)]
    """Following steps build L[m + 1][n + 1] in bottom up fashion
    Note: L[i][j] contains length of LCS of X[0..i-1]
    and Y[0..j-1]"""
    for i in range(m+1):
        for j in range(n+1):
            if i == 0 or j == 0 :
                L[i][j] = 0
            elif X[i-1] == Y[j-1]:
                L[i][j] = L[i-1][j-1]+1
            else:
                L[i][j] = 0
    return max ( max(l) for l in L )

def eval_expl(r, col):
  if re.search(re.compile(r'\bu[\'|\"]\s*\S+'), r[col]):
    return True
  return lcs(r["explanation"], r[col]) >= EXPL_EVAL_THR




for model_str in ['MM_dataset10k_Win5_NF32.keras'
 , 'MM_dataset10k_Win5_NF64.keras'
 , 'MM_dataset10k_Win5_NF128.keras']:
#  , 'MM_dataset10k_Win7_NF32.keras'
#  , 'MM_dataset10k_Win7_NF64.keras'
#  , 'MM_dataset10k_Win7_NF128.keras'
#  , 'MM_dataset10k_Win10_NF32.keras'
#  , 'MM_dataset10k_Win10_NF64.keras'
#  , 'MM_dataset10k_Win10_NF128.keras']:
  m = models.load_model(model_str)
  MAX_LEN = 100
  WIN_SIZE = int (model_str.split("_")[2][3:])
  EXPL_WIN = WIN_SIZE + 3
  EXPL_EVAL_THR = 3
  dataset = model_str.split("_")[1]+".csv"
  m.summary()

  print(WIN_SIZE)
  print(dataset)

  # Explain for dataset
  df = pd.read_csv(dataset, dtype={
      '__future__': 'object',
      'xrange': 'object'})
  df = df.drop ( df[df["lines of code"].isnull()].index )
  # df

  # consider only lines with annotated explanations
  df_expl = df[df["explanation"].notnull()]
  # add predictions
  df_expl = df_expl.join( pd.Series(name="pred", data = predict(df_expl["lines of code"]).reshape(-1), index=df_expl.index) )
  # consider only correct predictions
  df_expl["pred_class"] = df_expl["pred"].map(lambda x: 3 if x>0.5 else 2)
  df_expl = df_expl.loc[df_expl["pred_class"] == df_expl["class"]]



  # df_small = df_expl.iloc[:1000].copy()
  df_small = df_expl.copy()

  explainer = LimeTextExplainer(class_names=["py2", "py3"], split_expression=r"\s+")

  df_small['lime'] = df_small['lines of code'].map(explain)

  df_small["acc_expl_lime"] = df_small.apply(eval_expl, col="lime", axis=1)

  acc = sum(df_small["acc_expl_lime"])

  print (f"{acc}/{df_small.shape[0]}")
  print (m._name, f"{acc * 100 / df_small.shape[0]:.2f}")


Model: "MM_dataset10k_Win5_NF32"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 100, 8)               2048      ['input_1[0][0]']             
                                                                                                  
 conv1d_1 (Conv1D)           (None, 100, 32)              1312      ['embedding[0][0]']           
                                                                                                  
 conv1d (Conv1D)             (None, 100, 32)              1312      ['embedding[0][0]']           
                                                                            

Model: "MM_dataset10k_Win5_NF64"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, 100, 8)               2048      ['input_3[0][0]']             
                                                                                                  
 conv1d_5 (Conv1D)           (None, 100, 64)              2624      ['embedding_2[0][0]']         
                                                                                                  
 conv1d_4 (Conv1D)           (None, 100, 64)              2624      ['embedding_2[0][0]']         
                                                                            

Model: "MM_dataset10k_Win5_NF128"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 embedding_4 (Embedding)     (None, 100, 8)               2048      ['input_5[0][0]']             
                                                                                                  
 conv1d_9 (Conv1D)           (None, 100, 128)             5248      ['embedding_4[0][0]']         
                                                                                                  
 conv1d_8 (Conv1D)           (None, 100, 128)             5248      ['embedding_4[0][0]']         
                                                                           